In [1]:
%pip install pandas
%pip install numpy
%pip install itertools
%pip install matplotlib
%pip install seaborn
%pip install sklearn
%pip install BeautifulSoup4
%pip install requests
%pip install lxml
%pip install html5lib


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.5_1/libexec/bin/python -m pip install --upgrade pip
ERROR: No matching distribution found for itertools
Note: you may need to restart the kernel to use updated packages.
  Using cached contourpy-1.3.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadat

In [41]:
import pandas as pd
import numpy as np
from itertools import combinations

In [22]:
# Get compact mens and womens results + tourney seeds

df_reg_season = pd.concat(
    [pd.read_csv("../data/MRegularSeasonDetailedResults.csv").assign(League="M"),
    pd.read_csv("../data/WRegularSeasonDetailedResults.csv").assign(League="W")]
               )

df_tourney_seeds = pd.concat(
    [
        pd.read_csv("../data/MNCAATourneySeeds.csv").assign(League="M"),
        pd.read_csv("../data/WNCAATourneySeeds.csv").assign(League="W")
    ]

)

In [24]:
# print results
df_reg_season.head()


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,League
0,2003,10,1104,68,1328,62,N,0,27,58,...,16,22,10,22,8,18,9,2,20,M
1,2003,10,1272,70,1393,63,N,0,26,62,...,9,20,20,25,7,12,8,6,16,M
2,2003,11,1266,73,1437,61,N,0,24,58,...,14,23,31,22,9,12,2,5,23,M
3,2003,11,1296,56,1457,50,N,0,18,38,...,8,15,17,20,9,19,4,3,23,M
4,2003,11,1400,77,1208,71,N,0,30,61,...,17,27,21,15,12,10,7,1,14,M


In [25]:
# get tourney games for train + val data:

df_tourney_res =  pd.concat(
    [pd.read_csv("../data/MNCAATourneyDetailedResults.csv").assign(League="M"),
    pd.read_csv("../data/WNCAATourneyDetailedResults.csv").assign(League="W")]
               )

In [26]:
df_tourney_res.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,League
0,2003,134,1421,92,1411,84,N,1,32,69,...,14,31,17,28,16,15,5,0,22,M
1,2003,136,1112,80,1436,51,N,0,31,66,...,7,7,8,26,12,17,10,3,15,M
2,2003,136,1113,84,1272,71,N,0,31,59,...,14,21,20,22,11,12,2,5,18,M
3,2003,136,1141,79,1166,73,N,0,29,53,...,12,17,14,17,20,21,6,6,21,M
4,2003,136,1143,76,1301,74,N,1,27,64,...,15,20,10,26,16,14,5,8,19,M


In [51]:
#get all unique combinations of teamid and season to prepare for feature engineering of team data for teams in the tourney

tourney_teams = set()


w_team_ids = df_tourney_res[['WTeamID', 'Season']].drop_duplicates()
l_team_ids = df_tourney_res[['WTeamID', 'Season']].drop_duplicates()

team_ids = set()
for row in w_team_ids.itertuples(index=False, name=None):
    team_ids.add(row)
    
for row in l_team_ids.itertuples(index=False, name=None):
    team_ids.add(row)

team_ids = np.array(list(team_ids))

print(team_ids)

[[1411 2022]
 [1104 2023]
 [1328 2015]
 ...
 [1112 2009]
 [1272 2007]
 [1409 2003]]


In [ ]:
# extract data for each tournament teams
    # data: season, point_differential, win%, efg, TS%, OReb, DReb,  off efficiency, def efficiency, turnover ratio %, FTR

for id, season in team_ids: